## Setup

In [1]:
# set libraries to refresh
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import geopandas as gpd

In [3]:
from clustering.kmeans import custom_kmeans, get_oversized_clusters, run_optuna_kmeans_study, kmeans_secondpass #, parallel_kmeans_secondpass
from utils import plot_weights_vs_radii

## Load data

In [4]:
ROOT_DIR = Path("..")
DATA_DIR = ROOT_DIR / "data"
INPUT_DATA_DIR = DATA_DIR / "input"
OUTPUT_DATA_DIR = DATA_DIR / "output"

In [5]:
# data_type = "rooftops"
data_type = "grids"

# runs
n_jobs = 10
firstpass_n_trials = 50
secondpass_n_trials = 20

if data_type == "rooftops":

    gdf_for_cluster = gpd.read_parquet(INPUT_DATA_DIR / "rooftops.parquet")
    gdf_for_cluster.loc[:, "weight"] = 1
    # admin variables
    id_col = "rooftop_id"
    lat_col = "Lat_centroid"
    lon_col = "Lon_centroid"
    weight_col = "weight"
    epsg = 26191  # morocco
    # clustering variables
    desired_weight = 30
    desired_radius = 550
    secondpass_cutoff_weight = 50
    weight_importance_factor = 1

else:

    gdf_for_cluster = gpd.read_parquet(INPUT_DATA_DIR / "grids.parquet")
    # admin variables
    id_col = "grid_id"
    lat_col = "Lat"
    lon_col = "Lon"
    weight_col = "population"
    epsg = 3121  # philippines
    # clustering variables
    desired_weight = 240
    desired_radius = 1000
    weight_importance_factor = 1
    secondpass_cutoff_weight = 300

gdf_for_cluster.head()

,Lat,Lon,population,PSGC,urban,geometry,grid_id
25119277,8.0025,125.380000,0.144206,1102305007,True,POINT (125.38000 8.00250),GRID_25119278
25119278,8.0025,125.380833,0.146483,1102305007,True,POINT (125.38083 8.00250),GRID_25119279
25119279,8.0025,125.381666,0.140582,1102305007,True,POINT (125.38167 8.00250),GRID_25119280
25119280,8.0025,125.382500,0.144790,1102305007,True,POINT (125.38250 8.00250),GRID_25119281
25119281,8.0025,125.383333,0.032313,1102305007,True,POINT (125.38333 8.00250),GRID_25119282


## Clustering

- add option for dynamic radius selection back in

In [6]:
def cluster_data(gdf_for_cluster):

    if gdf_for_cluster[weight_col].sum() == 0:
        gdf_w_clusters = gdf_for_cluster.copy()
        gdf_w_clusters.loc[:, "cluster_id"] = "CLUSTER_0"
        gdf_w_clusters.loc[:, "cluster_weight"] = 0.0
        gdf_w_clusters.loc[:, "dense_area_guess"] = 0
        return gdf_w_clusters

    # first pass
    study_firstpass = run_optuna_kmeans_study(
        gdf=gdf_for_cluster,
        desired_cluster_weight=desired_weight,
        desired_cluster_radius=desired_radius,
        id_col=id_col,
        lat_col=lat_col,
        lon_col=lon_col,
        weight_col=weight_col,
        weight_importance_factor=desired_weight,
        epsg=epsg,
        n_trials=firstpass_n_trials,
        n_jobs=n_jobs,
        show_progress_bar=True,
    )

    # proper run with the best n_cluster
    clusters = custom_kmeans(
        df=gdf_for_cluster,
        n_clusters=study_firstpass.best_params["n_clusters"],
        id_col=id_col,
        lat_col=lat_col,
        lon_col=lon_col,
        weight_col=weight_col,
    )
    gdf_w_clusters = gdf_for_cluster.merge(clusters, on=id_col)
    gdf_w_clusters = gdf_w_clusters.sort_values(by="cluster_id")

    # second pass
    oversized_cluster_ids = get_oversized_clusters(
        gdf_w_clusters=gdf_w_clusters, cutoff_weight=secondpass_cutoff_weight
    )
    n_oversized = len(oversized_cluster_ids)
    print(f"Oversized clusters: {n_oversized}")

    # add urban_guess column
    gdf_w_clusters.loc[:, "dense_area_guess"] = 0
    gdf_w_clusters.loc[
        gdf_w_clusters["cluster_weight"] > secondpass_cutoff_weight,
        "dense_area_guess",
    ] = 1

    # second pass
    # if n_oversized > 0:
    #     # run re-clustering
    #     gdf_w_clusters = kmeans_secondpass(
    #         gdf_w_clusters=gdf_w_clusters,
    #         oversized_cluster_ids=oversized_cluster_ids,
    #         desired_cluster_weight=desired_weight,
    #         desired_cluster_radius=desired_radius,
    #         id_col=id_col,
    #         lat_col=lat_col,
    #         lon_col=lon_col,
    #         weight_col=weight_col,
    #         weight_importance_factor=weight_importance_factor,
    #         epsg=epsg,
    #         n_trials=secondpass_n_trials,
    #         n_jobs=n_jobs,
    #     )
    #     gdf_w_clusters = gdf_w_clusters.sort_values(by="cluster_id")

    return gdf_w_clusters

In [7]:
gdf_w_clusters = cluster_data(gdf_for_cluster)

  0%|          | 0/50 [00:00<?, ?it/s]

Oversized clusters: 22


In [ ]:
# single pass timings

# grids - 69,226 points different weights (smaller n_clusters)
# original: 17s

# rooftops - 17,367 points equal weight (much bigger n_clusters)
# original: 33s

In [ ]:
plot_weights_vs_radii(point_gdf_w_cluster=gdf_w_clusters, point_weight_col=weight_col, point_projected_epsg=epsg)

In [ ]:
gdf_w_clusters.plot(column="cluster_id", figsize=(5, 5))